In [1]:
import torch
import json
import time
import statistics
import numpy as np
import pickle as pkl

from tqdm import tqdm


from transformers import AutoTokenizer
from transformers.models.bloom.configuration_bloom import BloomConfig
from pruning.pruned_bloom import PrunedBloomForCausalLM
from node_attribution.utils import count_params
from rouge_score import rouge_scorer

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(f"bigscience/bloomz-1b1")

In [3]:
data = pkl.load(open("44_human_filtered_conv_pairs.pkl", "rb"))
cali_data = data[:22]
val_data = data[22:]

In [4]:
def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.pad_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.pad_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

In [5]:
def calc_p(data):
    perplexity_sum = 0
    for pair in tqdm(data):
        perplexity = score(sentence=pair, model=pruned_model, tokenizer=tokenizer)
        perplexity_sum += perplexity
    
    p = perplexity_sum / len(val_data)
    
    return p

In [6]:
weights_path = "pruned_30percent_560m_bloom.pt"
state_dict_shapes_path = "pruned_30percent_560m_bloom_state_dict_shapes.pkl"

bloom_config = BloomConfig(
    vocab_size=250880,
    hidden_size=1024,
    n_layer=24,
    n_head=16,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    use_cache=True,
    bos_token_id=1,
    eos_token_id=2,
    apply_residual_connection_post_layernorm=False,
    hidden_dropout=0.0,
    attention_dropout=0.0,
    pretraining_tp=1,  # TP rank used when training with megatron
    slow_but_exact=False,
    attention_softmax_in_fp32=True,
    bias_dropout_fusion=True,
    masked_softmax_fusion=True,
    offset_alibi=100,
    pad_token_id=3,
    seq_length=2048,
    skip_bias_add=True,
    skip_bias_add_qkv=False,
    unk_token_id=0,
    
)

In [7]:
pruned_model = PrunedBloomForCausalLM(bloom_config, state_dict_shapes_path)

In [8]:
pruned_model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [9]:
pruned_percent = 1.0 - (count_params(pruned_model)[-1] / 559214592)
print(pruned_percent)

0.30908961152430015


In [10]:
print(calc_p(cali_data))
print(calc_p(val_data))

  0%|                                                                                                                                                               | 0/22 [00:00<?, ?it/s]

0
torch.Size([57, 59, 256])
1
torch.Size([57, 59, 256])
2
torch.Size([57, 59, 256])
3
torch.Size([57, 59, 256])
4
torch.Size([57, 59, 256])
5
torch.Size([57, 59, 256])
6
torch.Size([57, 59, 256])
7
torch.Size([57, 59, 256])
8
torch.Size([57, 59, 256])
9
torch.Size([57, 59, 256])
10
torch.Size([57, 59, 256])
11
torch.Size([57, 59, 256])
12
torch.Size([57, 59, 256])
13
torch.Size([57, 59, 256])
14
torch.Size([57, 59, 320])
15
torch.Size([57, 59, 256])
16
torch.Size([57, 59, 256])
17
torch.Size([57, 59, 256])
18
torch.Size([57, 59, 320])
19
torch.Size([57, 59, 256])
20
torch.Size([57, 59, 704])
21
torch.Size([57, 59, 1024])
22
torch.Size([57, 59, 1024])
23
torch.Size([57, 59, 256])


  5%|██████▊                                                                                                                                                | 1/22 [00:03<01:09,  3.29s/it]

0
torch.Size([53, 55, 256])
1
torch.Size([53, 55, 256])
2
torch.Size([53, 55, 256])
3
torch.Size([53, 55, 256])
4
torch.Size([53, 55, 256])
5
torch.Size([53, 55, 256])
6
torch.Size([53, 55, 256])
7
torch.Size([53, 55, 256])
8
torch.Size([53, 55, 256])
9
torch.Size([53, 55, 256])
10
torch.Size([53, 55, 256])
11
torch.Size([53, 55, 256])
12
torch.Size([53, 55, 256])
13
torch.Size([53, 55, 256])
14
torch.Size([53, 55, 320])
15
torch.Size([53, 55, 256])
16
torch.Size([53, 55, 256])
17
torch.Size([53, 55, 256])
18
torch.Size([53, 55, 320])
19
torch.Size([53, 55, 256])
20
torch.Size([53, 55, 704])
21
torch.Size([53, 55, 1024])
22
torch.Size([53, 55, 1024])
23
torch.Size([53, 55, 256])


  9%|█████████████▋                                                                                                                                         | 2/22 [00:05<00:58,  2.92s/it]

...

0
torch.Size([123, 125, 256])
1
torch.Size([123, 125, 256])
2
torch.Size([123, 125, 256])
3
torch.Size([123, 125, 256])
4
torch.Size([123, 125, 256])
5
torch.Size([123, 125, 256])
6
torch.Size([123, 125, 256])
7
torch.Size([123, 125, 256])
8
torch.Size([123, 125, 256])
9
torch.Size([123, 125, 256])
10
torch.Size([123, 125, 256])
11
torch.Size([123, 125, 256])
12
torch.Size([123, 125, 256])
13
torch.Size([123, 125, 256])
14
torch.Size([123, 125, 320])
15
torch.Size([123, 125, 256])
16
torch.Size([123, 125, 256])
17
torch.Size([123, 125, 256])
18
torch.Size([123, 125, 320])
19
torch.Size([123, 125, 256])
20
torch.Size([123, 125, 704])
21
torch.Size([123, 125, 1024])
22
torch.Size([123, 125, 1024])
23
torch.Size([123, 125, 256])


 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 21/22 [02:11<00:12, 12.72s/it]

0
torch.Size([116, 118, 256])
1
torch.Size([116, 118, 256])
2
torch.Size([116, 118, 256])
3
torch.Size([116, 118, 256])
4
torch.Size([116, 118, 256])
5
torch.Size([116, 118, 256])
6
torch.Size([116, 118, 256])
7
torch.Size([116, 118, 256])
8
torch.Size([116, 118, 256])
9
torch.Size([116, 118, 256])
10
torch.Size([116, 118, 256])
11
torch.Size([116, 118, 256])
12
torch.Size([116, 118, 256])
13
torch.Size([116, 118, 256])
14
torch.Size([116, 118, 320])
15
torch.Size([116, 118, 256])
16
torch.Size([116, 118, 256])
17
torch.Size([116, 118, 256])
18
torch.Size([116, 118, 320])
19
torch.Size([116, 118, 256])
20
torch.Size([116, 118, 704])
21
torch.Size([116, 118, 1024])
22
torch.Size([116, 118, 1024])
23
torch.Size([116, 118, 256])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [02:23<00:00,  6.54s/it]

27434652.018668972


In [ ]:
num_trials = 20

In [ ]:
line = "Hello, I am a social bot! "
inputs = tokenizer(line, return_tensors="pt")
pruned_times = []

for i in range(num_trials):
    start = time.time()
    outputs = pruned_model.generate(
        input_ids=inputs["input_ids"], 
        max_new_tokens=100, 
        do_sample=True, 
        top_k=50, 
        top_p=0.95,
    )
    end = time.time()
    pruned_times.append(end - start)
    print(f"inference time: {end - start}")
    
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [ ]:
mean_pruned = np.mean(pruned_times)
std_pruned = np.std(pruned_times)
mode = statistics.median(pruned_times)
print(mean_pruned)
print(std_pruned)
print(mode)

In [ ]:
# while True:
#     line = input("You:")
#     inputs = tokenizer(line, return_tensors="pt")
#     outputs = pruned_model.generate(
#         input_ids=inputs["input_ids"], 
#         max_new_tokens=20, 
#         do_sample=True, 
#         top_k=50, 
#         top_p=0.95,
#     )
#     print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [ ]:
# full_line = "Person: My favorite movie is the The Day After Tomorrow\nSocialBot: Oh, interesting, I am not familiar with that movie! Can you tell me more about it?"
# prompt_line = "Person: My favorite movie is The Day After Tomorrow\nSocialBot: "
# completion = full_line.split(prompt_line)[-1]
# inputs = tokenizer(prompt_line, return_tensors="pt")

# #for i in range(num_trials):
# outputs = pruned_model.generate(
#     input_ids=inputs["input_ids"], 
#     max_new_tokens=25, 
#     do_sample=True, 
#     top_k=50, 
#     top_p=0.95,
# )
# out_seq = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# out_seq = out_seq[0].split("Person: My favorite movie is The Day After Tomorrow\nSocialBot: ")[-1]
# r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# rouge_scores = r_scorer.score(completion, out_seq)

In [ ]:
# from transformers import AutoTokenizer, BloomForCausalLM

In [ ]:
# model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [ ]:
# inputs = tokenizer("Person: My favorite movie is The Day After Tomorrow\nSocialBot: ", return_tensors="pt")
# start = time.time()
# outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=25, do_sample=True, top_k=50, top_p=0.95)
# end = time.time()
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# print(f"inference time: {end - start}")